In [15]:
%%writefile service.py
import web
import json
# import index

urls = (
    '/', 'index'
)

class index:
    #for testing purpose
    def GET(self):
        pyDict = {'test1':'works','test2':"works too"}
        web.header('Content-Type', 'application/json')
        return json.dumps(pyDict)

    def POST(self):
#         pyDict = {'one':1,'two':2}
        pyDict = web.input()
        web.header('Content-Type', 'application/json')
        return json.dumps(pyDict)

##ORIGINAL TEST CODE
# class index:
#     def GET(self):
#         return "Hello1, world!"
    
    
if __name__ == "__main__":
    app = web.application(urls, globals())
    app.run()

Overwriting service.py


In [7]:
!chmod +x service.py
!python service.py

http://0.0.0.0:8080/
^C


##Create Restful Web.py App

Adapted code from: https://gist.github.com/w0rm/3907294

In [8]:
%%writefile app.py
import web
import json
from controller import Controller
from baseline import Baseline

urls = (
    r'/resources(?:/(?P<resource_id>[0-9]+))?',
    'ResourceController',
)

class ResourceController(Controller):

    def list(self):
        return "list resources", format

    def baseline(self):
        request = json.loads(web.data())
        b = Baseline()
        respond = b.baseline_run(request)
        
        return respond
        
## ORIGINAL RESTFUL CONTROLLER
#     def get(self, resource_id):
#         return "retrieved resource", resource_id

#     def create(self):
#         resource = json.loads(web.data())
#         return "created resource", resource

#     def update(self, resource_id):
#         resource = json.loads(web.data())
#         return "updated resource", resource_id, resource

#     def delete(self, resource_id):
#         return "deleted resource", resource_id

app = web.application(urls, globals())

if __name__ == "__main__":
    app.run()

Overwriting app.py


In [2]:
%%writefile controller.py
import web

class Controller:

    methods = ("list", "get", "create", "update", "delete",
               "update_collection", "delete_collection")

    def __getattr__(self, name):
        if name in self.methods and "headers" in web.ctx:
            raise web.badrequest()
        else:
            raise AttributeError

    def POST(self, resource_id=None):
        if resource_id=="1":
            return self.baseline()
        elif resource_id=="2":
            return 'post_model2'
        else:
            raise web.badrequest()

## ORIGINAL CRUD OPERATION FROM RESTFUL CONTROLLER            
#     def GET(self, resource_id=None):
#         if resource_id is None:
#             return self.list()
#         else:
#             return self.get(resource_id)
#     #READ - TEST in browswer: http://localhost:8080/resources/23
        
#     def POST(self, resource_id=None):
#         if resource_id is None:
#             return self.create()
#         else:
#             raise web.badrequest()
#     #CREATE - TEST in terminal: curl -X POST  -H "Content-Type: application/json" -d '{"title":"a title","description":"a description","type":"a type","author":"an author"}' -i http://localhost:8080/resources/
    
#     def PUT(self, resource_id=None):
#         if resource_id is None:
#             return self.update_collection()
#         else:
#             return self.update(resource_id)
#     #UPDATE: TEST in terminal: curl -X PUT  -H "Content-Type: application/json" -d '{"id":"23", "title":"a title","description":"a description","type":"a type","author":"an author"}' -i http://localhost:8080/resources/23
    
#     def DELETE(self, resource_id=None):
#         if resource_id is None:
#             return self.delete_collection()
#         else:
#             return self.delete(resource_id)
#     #DELETE: TEST in terminal: curl -X DELETE http://localhost:8080/resources/23

Overwriting controller.py


In [22]:
!chmod +x app.py
!chmod +x controller.py



In [ ]:
!nohup python app.py 3000 &